In [1]:
# Environment setup

!sudo apt-get install -y pciutils
! sudo apt-get install zstd -y
!curl -fsSL https://ollama.ai/install.sh | sh

! pip install ollama
! pip install langchain-community
! pip install langchain-ollama
! pip install llama-index-llms-ollama
! pip install llama-index
! pip install langchain-community
! pip install langchain-text-splitters
! pip install langchain-huggingface
! pip install chromadb
! pip install langchain-chroma
! pip install langchain-ollama
! pip install sentence-transformers
! pip install unstructured
! pip install unstructured[pdf] nltk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 2 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (337 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: una

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found exi

In [2]:
! ollama serve > server.log 2>&1 &
! sleep 60s

In [3]:
! ollama pull qwen3-embedding
! ollama pull qwen3:1.7b

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader

import tempfile

In [5]:
sample_docs = [
    """
    Machine Learning Fundamentals

    Machine learning is a subset of artificial intelligence that enables systems to learn
    and improve from experience without being explicitly programmed. There are three main
    types of machine learning: supervised learning, unsupervised learning, and reinforcement
    learning. Supervised learning uses labeled data to train models, while unsupervised
    learning finds patterns in unlabeled data. Reinforcement learning learns through
    interaction with an environment using rewards and penalties.
    """,

    """
    Deep Learning and Neural Networks

    Deep learning is a subset of machine learning based on artificial neural networks.
    These networks are inspired by the human brain and consist of layers of interconnected
    nodes. Deep learning has revolutionized fields like computer vision, natural language
    processing, and speech recognition. Convolutional Neural Networks (CNNs) are particularly
    effective for image processing, while Recurrent Neural Networks (RNNs) and Transformers
    excel at sequential data processing.
    """,

    """
    Natural Language Processing (NLP)

    NLP is a field of AI that focuses on the interaction between computers and human language.
    Key tasks in NLP include text classification, named entity recognition, sentiment analysis,
    machine translation, and question answering. Modern NLP heavily relies on transformer
    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand
    context and relationships between words in text.
    """
]

In [6]:
tempdir = tempfile.mkdtemp()
for i in range(len(sample_docs)):
    with open(f"{tempdir}/doc_{i}.txt", "w") as f:
        f.write(sample_docs[i])

In [7]:
loader = DirectoryLoader(tempdir,
                         glob="*.txt",
                         loader_cls=TextLoader,
                         loader_kwargs={'autodetect_encoding': True})
documents = loader.load()
print(documents)

[Document(metadata={'source': '/tmp/tmpgl5o050k/doc_0.txt'}, page_content='\n    Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn\n    and improve from experience without being explicitly programmed. There are three main\n    types of machine learning: supervised learning, unsupervised learning, and reinforcement\n    learning. Supervised learning uses labeled data to train models, while unsupervised\n    learning finds patterns in unlabeled data. Reinforcement learning learns through\n    interaction with an environment using rewards and penalties.\n    '), Document(metadata={'source': '/tmp/tmpgl5o050k/doc_2.txt'}, page_content='\n    Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language.\n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis,\n    machine translation, and question answering. Mo

In [8]:
# document splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                               chunk_overlap=100,
                                               length_function=len,
                                               separators=[" "])
chunks = text_splitter.split_documents(documents)
print(len((chunks)))

5


In [9]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="qwen3-embedding",
)

persistent_directory = "./chromadb"
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings,
                                 persist_directory=persistent_directory,
                                    collection_name="rag_collection")

In [10]:
query = "what are the types of machine learning"
similar_docs = vectorstore.similarity_search(query, k=3)
print(similar_docs)

[Document(metadata={'source': '/tmp/tmpgl5o050k/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn\n    and improve from experience without being explicitly programmed. There are three main\n    types of machine learning: supervised learning, unsupervised learning, and reinforcement\n    learning. Supervised learning uses labeled data to train models, while unsupervised\n    learning finds patterns in unlabeled data. Reinforcement learning learns through\n    interaction'), Document(metadata={'source': '/tmp/tmpgl5o050k/doc_0.txt'}, page_content='learning finds patterns in unlabeled data. Reinforcement learning learns through\n    interaction with an environment using rewards and penalties.'), Document(metadata={'source': '/tmp/tmpgl5o050k/doc_1.txt'}, page_content='Deep Learning and Neural Networks\n\n    Deep learning is a subset of machine learning based on artificial neural networks.\

In [11]:
import langchain

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import ToolMessage
from langchain.tools import tool

langchain.debug = True

In [12]:
model = init_chat_model("qwen3:1.7b", model_provider="ollama")

In [13]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve relevant documentation to help answer user queries about LangChain."""
    # Retrieve top 4 most similar documents
    print(query)

    retrieved_docs = vectorstore.as_retriever().invoke(query, k=4)
    print(retrieved_docs)

    # Serialize documents for the model
    serialized = "\n\n".join(
        (f"Source: {doc.metadata.get('source', 'Unknown')}\n\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )


    # Return both serialized content and raw documents
    return serialized, retrieved_docs

In [14]:

def run_llm(query):
    """
    Run the RAG pipeline to answer a query using retrieved documentation.

    Args:
        query: The user's question

    Returns:
        Dictionary containing:
            - answer: The generated answer
            - context: List of retrieved documents
    """
    # Create the agent with retrieval tool
    system_prompt = (
        "You are a helpful AI assistant that answers questions about machine learning."
        "You have access to a tool that retrieves relevant documentation. "
        "Use the tool to find relevant information before answering questions. "
        "If you cannot find the answer in the retrieved documentation, reply exactly: Not enough information to answer the question."
        "Do not use your own knowledge"
    )

    agent = create_agent(model, tools=[retrieve_context], system_prompt=system_prompt)
    print(dir(agent))
    print(agent.get_prompts())
    # Build messages list
    messages = [{"role": "user", "content": query}]

    # Invoke the agent
    response = agent.invoke({"messages": messages})

    # Extract the answer from the last AI message
    answer = response["messages"][-1].content

    # Extract context documents from ToolMessage artifacts
    context_docs = []
    for message in response["messages"]:
        # Check if this is a ToolMessage with artifact
        if isinstance(message, ToolMessage) and hasattr(message, "artifact"):
            # The artifact should contain the list of Document objects
            if isinstance(message.artifact, list):
                context_docs.extend(message.artifact)

    return {
        "answer": answer,
        "context": context_docs
    }


In [15]:
prompt = "what is a python"
result = run_llm(prompt)

['InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abatch_with_config', '_abc_impl', '_acall_with_config', '_aprepare_state_snapshot', '_atransform_stream_with_config', '_batch_with_config', '_call_with_config', '_defaults', '_migrate_checkpoint', '_prepare_state_snapshot', '_repr_mimebundle_', '_transform_stream_with_config', 'abatch', 'abatch_as_completed', 'abulk_update_state', 'aclear_cache', 'aget_graph', 'aget_state', 'aget_state_history', 'aget_subgraphs', 'ainvoke', 'as_tool', 'assign', 'astream', 'astream_events',

In [16]:
print(result['answer'])
print("*"*50)
print(result['context'])

Not enough information to answer the question.
**************************************************
[Document(metadata={'source': '/tmp/tmpgl5o050k/doc_2.txt'}, page_content='Natural Language Processing (NLP)\n\n    NLP is a field of AI that focuses on the interaction between computers and human language.\n    Key tasks in NLP include text classification, named entity recognition, sentiment analysis,\n    machine translation, and question answering. Modern NLP heavily relies on transformer\n    architectures like BERT, GPT, and T5. These models use attention mechanisms to understand\n    context and relationships between words in text.'), Document(metadata={'source': '/tmp/tmpgl5o050k/doc_0.txt'}, page_content='Machine Learning Fundamentals\n\n    Machine learning is a subset of artificial intelligence that enables systems to learn\n    and improve from experience without being explicitly programmed. There are three main\n    types of machine learning: supervised learning, unsupervised l